In [ ]:
! pip install spacy
!python -m spacy download en_core_web_lg
import en_core_web_lg
nlp = en_core_web_lg.load()

from spacy.lang.en import English
parser = English()

In [3]:
import numpy as np 

In [ ]:
! pip install nltk
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
!pip install gensim
import gensim
from gensim import corpora
from gensim.models import Phrases

In [ ]:
import warnings, re
import string

In [ ]:
!pip install tqdm
import tqdm

In [ ]:
# -*- coding:utf-8 -*-
warnings.filterwarnings("ignore")

# Function processes each paragraph from the document and creates the List: 

# Entity type[key_word, doc_id, paragraph_id]


# Deleting urls and screen_names

In [ ]:
def delete_url(doc_paragraph):
    global URLS
    global SCREEN_NAME

    tokens = parser(doc_paragraph)
    for token in tokens:
      if token.orth_.isspace():
          continue
      elif token.like_url:
          URLS.append(token.text)
      elif token.orth_.startswith('@'):
          SCREEN_NAME.append(token.text)

# Extraction of Measures and Profiles using NER 

In [ ]:
def extract_ner(doc_paragraph, doc_id, paragraph_number):
    global dict_entities
    global Measures
    global Profiles
    global URLS
    global SCREEN_NAME

    sents = sent_tokenize(doc_paragraph)
    for sent in sents:
        nlp_sent = nlp(sent)
        for i in nlp_sent.ents:
            if str(i.label_) not in ['TIME','CARDINAL'] and i.text not in URLS and i.text not in SCREEN_NAME:
              if str(i.label_) in ['ORG', 'PERSON', 'NORP', 'LOC', 'GPE', 'PRODUCT']:
                  Profiles.append([i.text, doc_id, paragraph_number])
              elif str(i.label_) in 'PERCENT':
                  Measures.append(['percent', doc_id, paragraph_number])
              elif str(i.label_) in ['MONEY', 'ORDINAL']:
                  Measures.append([i.text, doc_id, paragraph_number])
              elif str(i.label_) in 'DATE':
                  Measures.append(['date', doc_id, paragraph_number])

    return Profiles, Measures

# Extraction of Profiles using rules

In [ ]:
def extract_profiles(doc_paragraph, doc_id, paragraph_number):
    global Profiles

    sents = sent_tokenize(doc_paragraph)
    for sent in sents:
       nlp_sent = nlp(sent)
       for token in nlp_sent:
          if (token.dep_ == "nsubj" and token.pos_ == "NOUN" or token.tag_ == "NN"
          or token.tag_ == "NNS" or token.tag_ == "NNP" or token.tag_ == "NNPS" 
          and token.text not in URLS and token.text not in [x[0] for x in Profiles] 
          and paragraph_number not in [x[2] for x in Profiles]) :
              chtok = [child for child in token.children]
              if len(chtok) >= 1:
                  if(chtok[0].pos_== "DET" or chtok[0].pos_=="PROPN"):
                      if len(str(token.text).lower()) > 2 and [str(token.text), doc_id, paragraph_number] not in Profiles:
                          Profiles.append([str(token.text), doc_id, paragraph_number])

# Extraction of Measures using rules

In [ ]:
def extract_measures(doc_paragraph, doc_id, paragraph_number):
    global Measures

    sents = sent_tokenize(doc_paragraph)
    for sent in sents:
       nlp_sent = nlp(sent)
       for token in nlp_sent:
          if ((token.dep_ == "amod" and token.head.pos_ == "NOUN" and token.head.dep_ != "nsubj") 
          or (token.dep_ == "compound" and token.head.pos_ == "NOUN" and token.head.dep_== "dobj") 
          and token.text not in URLS and token.text not in [x[0] for x in Measures] 
          and paragraph_number not in [x[2] for x in Measures]):
            if len(str(token.text).lower()) > 2:
                measure_kw = str(token.text).lower() + " " + str(token.head.text).lower()
                if [measure_kw, doc_id, paragraph_number] not in Measures:
                    Measures.append([measure_kw, doc_id, paragraph_number])

# Extraction of Actions using rules

In [ ]:
def extract_actions(doc_paragraph, doc_id, paragraph_number):
    global Actions
    action_kw = ''

    sents = sent_tokenize(doc_paragraph)
    for sent in sents:
      nlp_sent = nlp(sent)
      for token in nlp_sent:
        if token.dep_ == "dobj" and token.head.pos_ == "VERB" and token.head.tag_ != "VBG" and token.pos_ not in ["PRON", "DET", "SCONJ"] and token.head.dep_ != 'conj':
          child_list = [child for child in token.children]
          if len(child_list)==0 or child_list[0].dep_ not in  ["amod", "compound"]:
            action_kw = str(token.head.text).lower() + " " + str(token.text).lower()
          if action_kw != '' and [action_kw, doc_id, paragraph_number] not in Actions:
                Actions.append([action_kw, doc_id, paragraph_number])
        elif token.dep_ == "dobj" and token.head.lemma_ == 'do' and token.pos_ == "NOUN":
          action_kw = str(token.head.text).lower() + " " + str(token.text).lower()
          if action_kw != '' and [action_kw, doc_id, paragraph_number] not in Actions:
                Actions.append([action_kw, doc_id, paragraph_number])
        elif token.dep_ == "advcl" and token.head.pos_ == "VERB" and token.pos_ == "VERB":
          action_kw = str(token.head.text).lower() + " " + str(token.text).lower()
          if action_kw != '' and [action_kw, doc_id, paragraph_number] not in Actions:
                Actions.append([action_kw, doc_id, paragraph_number])
        elif (token.dep_ == "conj" and token.pos_ == "VERB" and token.head.pos_ == "VERB"):
              for head_child in token.head.children:
                 if head_child.dep_ == "cc":
                    action_kw = str(token.head.text).lower() + " " + str(head_child.text).lower() +" " + str(token.lemma_).lower()
                    for child in token.children:
                        if (child.dep_ == "dobj" and child.pos_ == "NOUN"):
                          action_kw = str(token.head.text).lower() + " " + str(head_child.text).lower() +" " + str(token.lemma_).lower() + " " + str(child.text).lower()
              if action_kw != '' and [action_kw, doc_id, paragraph_number] not in Actions:
                  Actions.append([action_kw, doc_id, paragraph_number])
        if token.dep_ == "dobj" and token.head.pos_ == "VERB" and token.head.dep_ == "csubj":
                  action_kw = str(token.head.text).lower() + " " + str(token.text).lower()
                  if action_kw != '' and [action_kw, doc_id, paragraph_number] not in Actions:
                      Actions.append([action_kw, doc_id, paragraph_number])
        elif (token.dep_ == "prep" and token.pos_ == "ADP" and token.head.pos_ == "VERB"):
            head_child_list = [child for child in token.children]
            if len(head_child_list) >= 1 and head_child_list[-1].dep_ != 'pobj':
                  for child in token.children:
                    if (child.dep_ == "pobj" and child.pos_ == "NOUN"):
                        goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(child.text).lower()
                        grandchildren_list = [grandchild for grandchild in child.children]
                        for grandchild in child.children:
                          if grandchild.dep_ == "compound" and grandchildren_list[0].pos_ == "DET":
                            action_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchildren_list[0].text).lower() + " " + str(grandchild.text) + " " + str(child.text)
                          elif grandchild.dep_ == "compound":
                            action_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchild.text) + " " + str(child.text)
            elif len(head_child_list) == 0:
              for child in token.children:
                    if (child.dep_ == "pobj" and child.pos_ == "NOUN"):
                        action_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(child.text).lower()
                        grandchildren_list = [grandchild for grandchild in child.children]
                        for grandchild in child.children:
                          if grandchild.dep_ == "compound" and grandchildren_list[0].pos_ == "DET":
                            action_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchildren_list[0].text).lower() + " " + str(grandchild.text) + " " + str(child.text)
                          elif grandchild.dep_ == "compound":
                            action_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchild.text) + " " + str(child.text)  
            if action_kw != '' and [action_kw, doc_id, paragraph_number] not in Actions:
                      Actions.append([action_kw, doc_id, paragraph_number])
        

# Extraction of Goals using rules

In [ ]:
def extract_goals(doc_paragraph, doc_id, paragraph_number):
    global Goals

    goal_kw = ''
    sents = sent_tokenize(doc_paragraph)
    for sent in sents:
        nlp_sent = nlp(sent)
        for token in nlp_sent:
          child_list = [child for child in token.children]
          if token.dep_ == "dobj" and token.head.pos_ == "VERB" and  token.pos_ not in ["SCONJ", "PRON", "PRON", "DET"]:
              goal_kw = str(token.head.text).lower() + " " + str(token.text).lower()
              for child in token.children:
                  if (len(child_list)>=2 and child.dep_ == "amod" 
                    and child_list[0].pos_ == "DET" 
                    and child_list[-1].dep_ == 'prep'):
                    for grandchild in child_list[-1].children:
                        if (grandchild.dep_ == "pobj"): 
                            goal_kw = str(token.head.text).lower() + " " + str(child_list[0].text).lower() + " " + str(child.text).lower() + " " + str(token.text).lower() + " " + str(child_list[-1].text).lower() + " " + str(grandchild.text).lower()
                  elif len(child_list)>=2 and child_list[0].dep_ == "det" and child_list[1].dep_ == "nmod":
                          goal_kw = str(token.head.text).lower() + " " + str(child_list[0].text).lower() + " " + str(child_list[1].text).lower()  + " " + str(token.text).lower()
                          grandchildren_list = [grandchild for grandchild in child.children]
                          for grandchild in child.children:
                              if len(grandchildren_list)>=1 and grandchild.dep_ == 'conj':
                                goal_kw = str(token.head.text).lower() + " " + str(child_list[0].text).lower() + " " + str(child_list[1].text).lower()  + " " +  str(grandchildren_list[0].text).lower() + " " + str(grandchildren_list[1].text).lower() + " " + str(token.text).lower()
                                ggrandchildren_list = [ggrandchild for ggrandchild in grandchild.children]
                                for gg in grandchild.children:
                                      if len(ggrandchildren_list)>=1 and gg.dep_ == "amod":
                                            goal_kw = str(token.head.text).lower() + " " + str(child_list[0].text).lower() + " " + str(child_list[1].text).lower()  + " " +  str(grandchildren_list[0].text).lower() + " " + str(gg.text).lower() + " " + str(grandchildren_list[1].text).lower() + " " + str(token.text).lower()
                  elif (len(child_list)>=1 and child.dep_ == "amod" and child_list[0].pos_ in ["DET", "NOUN"]):
                      goal_kw = str(token.head.text).lower() + " " + str(child_list[0].text).lower() + " " + str(child.text) + " " + str(token.text).lower()
                  elif (child.dep_ == "amod"): 
                      goal_kw = str(token.head.text).lower() + " " + str(child.text) + " " + str(token.text).lower()
                  elif (len(child_list)>=1 and child.dep_ == "compound" and child_list[0].pos_ in ["DET", "NOUN"]):
                      goal_kw = str(token.head.text).lower() + " " + str(child_list[0].text).lower() + " " + str(token.text).lower()
                  elif (child.dep_ == "compound"):
                      for grandchild in child.children:
                        if (grandchild.dep_ == "compound"):
                          goal_kw = str(token.head.text).lower() + " " + str(grandchild.text)+ " " + str(child.text) + " " + str(token.text).lower()
                        elif (grandchild.dep_ == "amod"):
                          goal_kw = str(token.head.text).lower() + " " + str(grandchild.text)+ " " + str(child.text) + " " + str(token.text).lower()
              if goal_kw != '' and [goal_kw, doc_id, paragraph_number] not in Goals and [goal_kw, doc_id, paragraph_number] not in Actions:
                  Goals.append([goal_kw, doc_id, paragraph_number])
          elif token.dep_ == "xcomp" and token.head.pos_ == "VERB":
              for child in token.children:
                  if child.dep_ == "dobj":
                    goal_kw = str(token.text).lower() + " " + str(child.text).lower()
                    for grandchild in child.children:
                      if grandchild.dep_ == "amod":
                          goal_kw = str(token.text).lower() + " " + str(grandchild.text).lower() + " " + str(child.text).lower()
              if goal_kw != '' and [goal_kw, doc_id, paragraph_number] not in Goals and [goal_kw, doc_id, paragraph_number] not in Actions:
                    Goals.append([goal_kw, doc_id, paragraph_number])
          elif (token.dep_ == "prep" and token.pos_ == "ADP" and token.head.pos_ == "VERB"):
            head_child_list = [child for child in token.children]
            if len(head_child_list) >= 1 and head_child_list[-1].dep_ != 'pobj':
                  for child in token.children:
                    if (child.dep_ == "pobj" and child.pos_ == "NOUN"):
                        goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(child.text).lower()
                        grandchildren_list = [grandchild for grandchild in child.children]
                        for grandchild in child.children:
                          if grandchild.dep_ == "compound" and grandchildren_list[0].pos_ == "DET":
                            goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchildren_list[0].text).lower() + " " + str(grandchild.text) + " " + str(child.text)
                          elif grandchild.dep_ == "compound":
                            goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchild.text) + " " + str(child.text)
            elif len(head_child_list) == 0:
              for child in token.children:
                    if (child.dep_ == "pobj" and child.pos_ == "NOUN"):
                        goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(child.text).lower()
                        grandchildren_list = [grandchild for grandchild in child.children]
                        for grandchild in child.children:
                          if grandchild.dep_ == "compound" and grandchildren_list[0].pos_ == "DET":
                            goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchildren_list[0].text).lower() + " " + str(grandchild.text) + " " + str(child.text)
                          elif grandchild.dep_ == "compound":
                            goal_kw = str(token.head.text).lower() + " " + str(token.text).lower() + " " + str(grandchild.text) + " " + str(child.text)  
            if goal_kw != '' and [goal_kw, doc_id, paragraph_number] not in Goals and [goal_kw, doc_id, paragraph_number] not in Actions:
                      Goals.append([goal_kw, doc_id, paragraph_number])

# Extraction of Categories on the paragraph level using Gensim

In [ ]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [ ]:
def prepare_text_for_lda(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop and token not in URLS]
    #tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
def extract_categories(doc_paragraph, doc_id, paragraph_number):
    global Categories
    global URLS
    
    cleaned_sents = []
    sentences = sent_tokenize(doc_paragraph.lower())
    for sent in  sentences:
        cleaned_sents.append(prepare_text_for_lda(sent))
    if len(cleaned_sents)>5:
        dictionary = corpora.Dictionary(cleaned_sents)
        corpus = [dictionary.doc2bow(text) for text in cleaned_sents]
        NUM_TOPICS = 3
        num_words = 3
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        for index, topic in ldamodel.show_topics(num_words=num_words, formatted=False):
            if len(topic)>0:
                for w in topic:
                  tokens = nlp(w[0])
                  for tok in tokens:
                    if tok.pos_ in ["NOUN", "PROPN"] and w[1]>=0.03:
                        if [w[0], doc_id, paragraph_number] not in Categories and w[0] not in URLS:
                            Categories.append([w[0], doc_id, paragraph_number])

# Run extraction functions for all the keywords

In [ ]:
dict_entities = {}
Measures = []
Actions = []
Profiles = []
Goals = []
Categories = []
URLS =[]
SCREEN_NAME = []


# Results

In [ ]:
import os

def kw_extraction(folder_path):
    doc_id = 1
    for files in os.walk(folder_path):
      file_list = files[2]
      for file_name in file_list[0:1]:
        print(file_name)
        file_path = files[0] + "/" + file_name
        with open (file_path, 'r', encoding='utf-8') as doc:
          paragraph_number = 0
          file_text = doc.readlines()
          if len(file_text)>1:
            paragraph_number = 1
            for line in file_text:
                delete_url(line)
                extract_ner(line, doc_id, paragraph_number)
                extract_measures(line, doc_id, paragraph_number)
                extract_profiles(line, doc_id, paragraph_number)
                extract_actions(line, doc_id, paragraph_number)
                extract_goals(line, doc_id, paragraph_number)
                extract_categories(line, doc_id, paragraph_number)
                paragraph_number += 1
          doc_id += 1

kw_extraction('/content/drive/Shareddrives/SemanticGraph/Semantic graph/data_set')

print('Measures: ', len(Measures))
print('Profiles: ', len(Profiles))
print('Actions: ', len(Actions))
print('Goals: ', len(Goals))
print('Categories: ', len(Categories))
print('Measures: ', Measures)
print('Profiles: ', Profiles)
print('Actions: ', Actions)
print('Goals: ', Goals)
print('Categories: ', Categories)
print('URLS: ', URLS)
print('SCREEN_NAME: ', SCREEN_NAME)

doc__2
Measures:  693
Profiles:  1196
Actions:  297
Goals:  230
Categories:  48
Measures:  [['date', 1, 2], ['many ways', 1, 2], ['different stages', 1, 2], ['only certainty', 1, 2], ['first', 1, 3], ['date', 1, 3], ['date', 1, 3], ['date', 1, 3], ['complex world', 1, 3], ['largest companies', 1, 3], ['date', 1, 4], ['groundbreaking study', 1, 4], ['global growth', 1, 4], ['economic growth', 1, 4], ['whole story', 1, 4], ['real fears', 1, 5], ['overturned marketplace', 1, 5], ['entire organization', 1, 5], ['digital transformation', 1, 6], ['new demands', 1, 6], ['corporate purpose', 1, 6], ['many problems', 1, 6], ['expedited timeframe', 1, 7], ['important trends', 1, 7], ['third', 1, 9], ['date', 1, 9], ['date', 1, 9], ['date', 1, 9], ['date', 1, 9], ['date', 1, 9], ['independent firms', 1, 9], ['other firm', 1, 9], ['third parties', 1, 9], ['such authority', 1, 9], ['swiss entity', 1, 9], ['key findings', 1, 9], ['unique lens', 1, 9], ['evolving attitudes', 1, 9], ['full impact', 1,

# Creation of the key-words dictionary 

In [ ]:
def dict_creation(Measures, Profiles, Actions, Goals, Categories):
  global dict_entities
  dict_entities["Measures"] = Measures
  dict_entities["Profiles"] = Profiles
  dict_entities["Actions"] = Actions
  dict_entities["Goals"] = Goals
  dict_entities["Categories"] = Categories
  np.save('/content/drive/Shareddrives/SemanticGraph/Semantic graph/extracted_ents.npy', dict_entities)
  return dict_entities

In [ ]:
dict_creation(Measures, Profiles, Actions, Goals, Categories)
for key in dict_entities.keys():
  print(key, dict_entities[key])
  print(len(dict_entities[key]))

Measures [['date', 1, 2], ['many ways', 1, 2], ['different stages', 1, 2], ['only certainty', 1, 2], ['first', 1, 3], ['date', 1, 3], ['date', 1, 3], ['date', 1, 3], ['complex world', 1, 3], ['largest companies', 1, 3], ['date', 1, 4], ['groundbreaking study', 1, 4], ['global growth', 1, 4], ['economic growth', 1, 4], ['whole story', 1, 4], ['real fears', 1, 5], ['overturned marketplace', 1, 5], ['entire organization', 1, 5], ['digital transformation', 1, 6], ['new demands', 1, 6], ['corporate purpose', 1, 6], ['many problems', 1, 6], ['expedited timeframe', 1, 7], ['important trends', 1, 7], ['third', 1, 9], ['date', 1, 9], ['date', 1, 9], ['date', 1, 9], ['date', 1, 9], ['date', 1, 9], ['independent firms', 1, 9], ['other firm', 1, 9], ['third parties', 1, 9], ['such authority', 1, 9], ['swiss entity', 1, 9], ['key findings', 1, 9], ['unique lens', 1, 9], ['evolving attitudes', 1, 9], ['full impact', 1, 9], ['new reality', 1, 9], ['potential pool', 1, 9], ['short term', 1, 9], ['risk